# Importing dependencies

In [2]:
from deepface import DeepFace
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib as plt
import math
import glob
import time 
#I've added timers at the most computationally expensive places so that you can gauge how much detail is feasible for you to achieve depending on your hardware

# Splitting the relevant videos into frames

**In here, you can set:**
* the frames/second rate
* the input folder
* the output folder
* the naming system you want for your output

In [3]:
#For some reason he doesn't like backslashes when specifying file paths so make sure you use frontslashes and put an 'r' infront of the path
#Also doesn't like it if your path is not on your local machine (e.g. a OneDrive file)

# #This line starts a timer
# start = time.time()

pathOut = r'C:/Users/chazzers/Desktop/Recordings/Frames/'
count = 0
counter = 1
listing = os.listdir(r'C:/Users/chazzers/Desktop/Recordings/Charles')
for vid in listing:
    vid = r'C:/Users/chazzers/Desktop/Recordings/Charles/' + vid
    cap = cv2.VideoCapture(vid)
    count = 0
    counter += 1
    success = True
    while success:
        success,image = cap.read()
#         print('read a new frame:',success)
        if count%1000 == 0 :
              cv2.imwrite(pathOut + 'video%d'%counter + 'frame%d.jpg'%count ,image)
        count+=1

#change the number after modulo sign to get the desired frame rate

# #This line stops the timer
# elapsed_time_split = (time.time() - start) 

Dividing the total elapsed time by the number images split to get the time to extract one frame from the video

In [22]:
splitsec = elapsed_time_split/count
print(splitsec)

* Use the code below to split videos into *ALL* its frames -> Not deemed necessary unless one wants to start looking at mircoexpressions. Currently 2 images per second is more than sufficient to provide results in line with external measures of engagement/ affect

In [23]:
# #splitting the video into individual frames

# # Playing video from file:
# cap = cv2.VideoCapture(test_video)

# try:
#     if not os.path.exists('video_frames'):
#         os.makedirs('video_frames')
# except OSError:
#     print ('Error: Creating directory of data')

# currentFrame = 0
# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     # Saves image of the current frame in jpg file
#     name = './test_videodata/frame' + str(currentFrame) + '.jpg'
#     print ('Creating...' + name)
#     cv2.imwrite(name, frame)

#     # To stop duplicate images
#     currentFrame += 1

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

---

# Analyzing frames

* Change the filename to include the name of the relevant video below
* i.e. r'C:/Users/chazzers/Desktop/Recordings/Frames/video2.jpg' if you want video 2 
* or r'C:/Users/chazzers/Desktop/Recordings/Frames/video3.jpg' if you want video 3 etc.
* or r'C:/Users/chazzers/Desktop/Recordings/Frames/.jpg' if you want all the videos in one go.

Putting image frames into an array so that DeepFace can run on them

In [4]:
# # #This line starts a timer
# # start = time.time()

img_array2 = []
for filename in glob.glob(r'C:/Users/chazzers/Desktop/Recordings/Frames/video2*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array2.append(img)
    
# # #This line stops the timer    
# # elapsed_time_fl = (time.time() - start) 

Dividing the total elapsed time by the number images appended to the array to get the time to extract append one image to the array

In [6]:
# arraysec = elapsed_time_fl/len(img_array)
# print(arraysec)

In [7]:
#Running deepface on the array just created
df_test2 = DeepFace.analyze(img_path=img_array2, actions = ['emotion'],enforce_detection=False)
# Alos possible to do 'age', 'gender', 'race' but increases the amount of time taken massively

Analyzing: 100%|███████████████████████████████████████████████████████████████████| 1875/1875 [08:38<00:00,  3.62it/s]


In [8]:
# This returns a python dictionary of emotions
#unhash to check
# df_test2

# Inserting the data into a Pandas dataframe so we can manipulate it

In [9]:
import pandas as pd

data = df_test2

emotions = set()
# First we need to find out all unique emotions
for key, value in data.items():
    for emotion in value['emotion'].keys():
        emotions.add(emotion)

rows = []
columns = ['instance'] + list(emotions)

for key, value in data.items():
    rows.append([0] * len(columns)) # Start creating a new row with zeros

    key = key.split('_')[1] # Get the 1,2,3 out of the instance
    rows[-1][0] = key
    for emotion, emotion_value in value['emotion'].items():
        rows[-1][columns.index(emotion)] = emotion_value # place the emotion in the correct index


df = pd.DataFrame(rows, columns=columns)
df.set_index('instance', inplace=True)

# print(df)

# Saving it to a csv file

(not strictly necessary, but easier to share data later on with excel users)

In [10]:
import tkinter as tk
from tkinter import filedialog

Setting the file path to which the excel file can be exported

In [11]:
export_file_path = r'C:/Users/chazzers/Desktop/Recordings/Results'

In [12]:
root= tk.Tk()

canvas1 = tk.Canvas()
canvas1.pack()

def exportCSV ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    df.to_csv (export_file_path, index = True, header=True)

saveAsButton_CSV = tk.Button(text='Export CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)

root.mainloop()